In [ ]:

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'indian-food-101:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F865197%2F1526436%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240319%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240319T221333Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3c1a4178377cebaa3e2fc0f7bad27f8457bccd61176b17e981282d033a43ba90e2eb18caecc0cc60c624e0674ca2a4e64becaf6d4f7168a800c79503ff31736381b247859cd605cbfd60153c4f704f17c1892d1b4a666090aaacacf2e028411ca9603cf43e5caf17a52c0b75f3c0d9c665bc8b17922410be25e66928afc7f85745aa5eed34e39c43b9b66beeeecd7d54493957733a0a6d2b4b4b6121f58ed65b977a1f284365cdcf02930319e830d229c10fe870396d765826512ea58fce08052a215676b458703d45ba4bf002ba4c3cdbbd8ddf6963d8f8913283abf264aab53ccc34a5cd0755dc86efc4e98e42020873493fc1efd2939eb997e2730962d12d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 6951 bytes downloaded
Downloaded and uncompressed: indian-food-101
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-food-101/indian_food.csv


In [ ]:
data=pd.read_csv("/kaggle/input/indian-food-101/indian_food.csv")
# data.head(10)

NameError: name 'pd' is not defined

In [ ]:
# Unique values for each columns

# print(pd.unique(data.ingredients))
# print(pd.unique(data.flavor_profile))
# print(pd.unique(data.course))
# print(pd.unique(data.diet))
# print(pd.unique(data.state))
### data.loc[data['state'] == '-1']

In [ ]:
# Cleaning Data
# data.loc[data['flavor_profile']== '-1']
# data.loc[data['name']=='Copra paak','flavor_profile']
data.loc[data['name'].isin(['Copra paak','Puttu','Kansar']),'flavor_profile']='sweet'
data.loc[data['flavor_profile'] == '-1', 'flavor_profile'] = "other"


In [ ]:
# Cleaned values
data.tail(10)

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
245,Pani Pitha,"Tea leaves, white sesame seeds, dry coconut, s...",vegetarian,10,20,other,main course,Assam,North East
246,Payokh,"Basmati rice, rose water, sugar, clarified but...",vegetarian,-1,-1,sweet,dessert,Assam,North East
247,Prawn malai curry,"Coconut milk, prawns, garlic, turmeric, sugar",non vegetarian,15,50,spicy,main course,West Bengal,East
248,Red Rice,"Red pepper, red onion, butter, watercress, oli...",vegetarian,-1,-1,other,main course,-1,-1
249,Shukto,"Green beans, bitter gourd, ridge gourd, banana...",vegetarian,10,20,spicy,main course,West Bengal,East
250,Til Pitha,"Glutinous rice, black sesame seeds, gur",vegetarian,5,30,sweet,dessert,Assam,North East
251,Bebinca,"Coconut milk, egg yolks, clarified butter, all...",vegetarian,20,60,sweet,dessert,Goa,West
252,Shufta,"Cottage cheese, dry dates, dried rose petals, ...",vegetarian,-1,-1,sweet,dessert,Jammu & Kashmir,North
253,Mawa Bati,"Milk powder, dry fruits, arrowroot powder, all...",vegetarian,20,45,sweet,dessert,Madhya Pradesh,Central
254,Pinaca,"Brown rice, fennel seeds, grated coconut, blac...",vegetarian,-1,-1,sweet,dessert,Goa,West


from matplotlib import pyplot as plt
_df_0['prep_time'].plot(kind='hist', bins=20, title='prep_time')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['cook_time'].plot(kind='hist', bins=20, title='cook_time')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('diet').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('flavor_profile').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['flavor_profile'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='prep_time', y='flavor_profile', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['course'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='prep_time', y='course', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['state'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='prep_time', y='state', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
# Performing One hot encoding
discrete_df = pd.get_dummies(data,columns=['ingredients','diet','flavor_profile'])
discrete_df

,name,prep_time,cook_time,course,state,region,"ingredients_Aloo, tomatoes, mustard oil, bay leaf, cinnamon stick","ingredients_Aloo, urad dal, mustard, ginger, curry leaves","ingredients_Amaranth leaves, split urad dal, mustard seeds, grated coconut, red chili","ingredients_Apricots, sugar syrup",...,"ingredients_Yogurt, fresh coconut, sesame seeds, semolina, gram flour","ingredients_Yogurt, ginger, curry leaves, baking soda, green chilli","ingredients_Yogurt, milk, nuts, sugar",diet_non vegetarian,diet_vegetarian,flavor_profile_bitter,flavor_profile_other,flavor_profile_sour,flavor_profile_spicy,flavor_profile_sweet
0,Balu shahi,45,25,dessert,West Bengal,East,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,Boondi,80,30,dessert,Rajasthan,West,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,Gajar ka halwa,15,60,dessert,Punjab,North,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,Ghevar,15,30,dessert,Rajasthan,West,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,Gulab jamun,15,40,dessert,West Bengal,East,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,Til Pitha,5,30,dessert,Assam,North East,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
251,Bebinca,20,60,dessert,Goa,West,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
252,Shufta,-1,-1,dessert,Jammu & Kashmir,North,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
253,Mawa Bati,20,45,dessert,Madhya Pradesh,Central,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [ ]:
target= data['course']
df= discrete_df.drop(columns=['course','state','region','name'],axis=1)
df

,prep_time,cook_time,"ingredients_Aloo, tomatoes, mustard oil, bay leaf, cinnamon stick","ingredients_Aloo, urad dal, mustard, ginger, curry leaves","ingredients_Amaranth leaves, split urad dal, mustard seeds, grated coconut, red chili","ingredients_Apricots, sugar syrup","ingredients_Arbi ke patte, sesame seeds, gur, bengal gram flour, imli","ingredients_Arhar dal, coconut oil, curry leaves, mustard seeds, red chilli","ingredients_Arhar dal, sambar powder, tomato, curry leaves, fennel seeds","ingredients_Baby corn, french beans, garam masala, ginger, carrot",...,"ingredients_Yogurt, fresh coconut, sesame seeds, semolina, gram flour","ingredients_Yogurt, ginger, curry leaves, baking soda, green chilli","ingredients_Yogurt, milk, nuts, sugar",diet_non vegetarian,diet_vegetarian,flavor_profile_bitter,flavor_profile_other,flavor_profile_sour,flavor_profile_spicy,flavor_profile_sweet
0,45,25,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,80,30,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,15,60,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,15,30,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,15,40,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,5,30,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
251,20,60,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
252,-1,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
253,20,45,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [ ]:
# Converting target string to numeric labels

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(data['course'])
label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       0, 2, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 3, 2, 1, 1, 1, 2, 2, 2, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 0, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 2, 2, 2,
       1, 1, 2, 2, 1, 2, 0, 1, 0, 2, 2, 0, 1, 1, 1, 2, 2, 2, 1, 1, 1, 0,
       2, 2, 1, 1, 2, 1, 1, 2, 0, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0])

In [ ]:
# Spliting data inro training set and testing set

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(df,label, test_size=0.2)

In [ ]:
# implementing classification algorithm
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train, Y_train)
ypred = model.predict(X_test)

In [ ]:
# Checking accuracy
accuracy = model.score(X_test, Y_test)
print(accuracy)
from sklearn.metrics import classification_report
print(classification_report(Y_test, ypred))

0.8235294117647058
              precision    recall  f1-score   support

           0       0.89      1.00      0.94        17
           1       0.79      0.88      0.83        25
           2       0.75      0.33      0.46         9

    accuracy                           0.82        51
   macro avg       0.81      0.74      0.75        51
weighted avg       0.82      0.82      0.80        51



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

clf = RandomForestClassifier(n_estimators=2 ,min_samples_split=3 , min_samples_leaf=2)
clf.fit(X_train,Y_train)
pred_clf = clf.predict(X_test)
# sklearn.metrics.accuracy_score(Y_test,pred_clf)

In [ ]:
# pred_clf

accuracy = clf.score(X_test, Y_test)
print(accuracy)
print(classification_report(Y_test, pred_clf))

0.6078431372549019
              precision    recall  f1-score   support

           0       0.61      1.00      0.76        17
           1       0.61      0.56      0.58        25
           2       0.00      0.00      0.00         9

    accuracy                           0.61        51
   macro avg       0.41      0.52      0.45        51
weighted avg       0.50      0.61      0.54        51



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {"n_estimators":[2,5,10,20],"min_samples_split":[2,3],"min_samples_leaf":[1,2,3]}
grid_search = GridSearchCV(estimator=clf,param_grid=param_grid)
grid_search.fit(X_train,Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(estimator=RandomForestClassifier(min_samples_leaf=2,
                                              min_samples_split=3,
                                              n_estimators=2),
             param_grid={'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 3],
                         'n_estimators': [2, 5, 10, 20]})

In [ ]:
grid_search.best_params_

{'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 10}

In [ ]:
clf = RandomForestClassifier(n_estimators=20 ,min_samples_split=2 , min_samples_leaf=1)
clf.fit(X_train,Y_train)
pred_clf = clf.predict(X_test)


In [ ]:
accuracy = clf.score(X_test, Y_test)
print(accuracy)
print(classification_report(Y_test, pred_clf))

0.7843137254901961
              precision    recall  f1-score   support

           0       0.89      1.00      0.94        17
           1       0.72      0.92      0.81        25
           2       0.00      0.00      0.00         9

    accuracy                           0.78        51
   macro avg       0.54      0.64      0.58        51
weighted avg       0.65      0.78      0.71        51



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
pipe_lr = Pipeline([('minmax',MinMaxScaler()),('lr',LogisticRegression())])
pipe_lr.fit(X_train,Y_train)
score = pipe_lr.score(X_test,Y_test)
score

0.7843137254901961